In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.declarative import declarative_base

In [2]:
# Import the census data into a pandas DataFrame
# Import cities file as DataFrame
measurements_pd = pd.read_csv("python-challenge/surfs_up/hawaii_measurements_clean.csv")
measurements_pd.head(10)
# Preview the data

,station,date,prcp,tobs
0,USC00519397,1/1/2010,0.08,65
1,USC00519397,1/2/2010,0.00,63
2,USC00519397,1/3/2010,0.00,74
3,USC00519397,1/4/2010,0.00,76
4,USC00519397,1/7/2010,0.06,70
5,USC00519397,1/8/2010,0.00,64
6,USC00519397,1/9/2010,0.00,68
7,USC00519397,1/10/2010,0.00,73
8,USC00519397,1/11/2010,0.01,64
9,USC00519397,1/12/2010,0.00,61


In [3]:
# Import the census data into a pandas DataFrame
# Import cities file as DataFrame
stations_pd = pd.read_csv("python-challenge/surfs_up/hawaii_stations_clean.csv")
stations_pd.head()

# Preview the data

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [4]:
# Declare column types and import the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [5]:
# Declare base in SQL Alchemy
Base = declarative_base()

In [6]:
# Create Engine
### BEGIN SOLUTION
engine = create_engine("sqlite:///hawaii.sqlite")
### END SOLUTION

In [10]:
# Creates Classes for db Tables
class Measurements(Base):
    __tablename__ = 'Measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(String(255))
    prcp = Column(String(255))
    tobs = Column(String(255))
    
class Stations(Base):
    __tablename__ = 'Stations'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    latitude = Column(String(255))
    longitude = Column(String(255))
    elevation = Column(String(255))
    location = Column(String(255))              

C:\Users\craig\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Measurements, and will be replaced in the string-lookup table.
  item.__name__


InvalidRequestError: Table 'Measurements' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [11]:
# Create tables associated with the classes.
Base.metadata.create_all(engine)

In [12]:
# Verify tables are made
engine.table_names()

['Measurements', 'Stations']

In [14]:
# Load dataframe into tables
def load_table(engine, table, csvfile):
    # connect to the database
    conn = engine.connect()
    
    # Load the CSV file into a pandas dataframe 
    df_of_data_to_insert = pd.read_csv(csvfile)
    data = df_of_data_to_insert.to_dict(orient='records')

    # Delete  rows in the table 
    conn.execute(table.delete())

    # Load  dataframe into the db
    conn.execute(table.insert(), data)
    
# Load data for each table
load_table(engine, Measurements.__table__, 'python-challenge/surfs_up/hawaii_measurements_clean.csv')
load_table(engine, Stations.__table__, 'python-challenge/surfs_up/hawaii_stations_clean.csv')


In [17]:
# Use a basic query to validate that the data was inserted correctly for table `measurement`
engine.execute("SELECT * FROM Measurements LIMIT 5").fetchall()

[(1, 'USC00519397', '1/1/2010', '0.08', '65'),
 (2, 'USC00519397', '1/2/2010', '0.0', '63'),
 (3, 'USC00519397', '1/3/2010', '0.0', '74'),
 (4, 'USC00519397', '1/4/2010', '0.0', '76'),
 (5, 'USC00519397', '1/7/2010', '0.06', '70')]

In [18]:
# Use a basic query to validate that the data was inserted correctly for table `measurement`
engine.execute("SELECT * FROM Stations LIMIT 5").fetchall()

[(1, 'USC00519397', '21.2716', '-157.8168', '3.0', None),
 (2, 'USC00513117', '21.4234', '-157.8015', '14.6', None),
 (3, 'USC00514830', '21.5213', '-157.8374', '7.0', None),
 (4, 'USC00517948', '21.3934', '-157.9751', '11.9', None),
 (5, 'USC00518838', '21.4992', '-158.0111', '306.6', None)]